# Compte rendu Du Projet SAE 15
### Par **HARTMANN** Matthias et **BIENVENU** Victor
Dans le cadre du Projet de SAE 15, nous avons dû produire un outil d'analyse de l'activité d'un serveur.
Vous retrouverez cet outil dans ce notebook python.
Dans un premier temps vous verrez l'ensembles des fonctions pythons qui ont été écrites pour cet outil et elles vous seront commentées.
Ensuite vous pourrez accédez à l'outil qui vous permettra de visualiser de plusieurs façons les activités suivantes :
- Le nombre et les ports ouverts sur le serveur
- La connectivité du serveur par rapport à un autre serveur.


## Partie 1 : Les fonctions pythons

### Importation des modules permettant l'exploitations des fichiers CSV et la génération de graphiques

In [1]:
import csv
from typing import Dict, List
import matplotlib.pyplot as plt
from dateutil import parser
from datetime import datetime
import sys
import ipywidgets as widgets
from IPython.display import display, clear_output


### Fonctions qui permettent de gérer les dates et les timestamps (Utilisé par les deux métriques) 

In [2]:
def dateFormatter(epoch: int, _=None) -> str:
    """!
    @brief Cette fonction permet de convertir un timestamp epoch en une date compréhensible.

    Paramètres : 
        @param epoch : int => timestamp dans le format epoch
        @param _ = None => Un paramètre qui est requis par matplotlib mais non utilisé par la fonction
    Retour de la fonction : 
        @return str => La date compréhensible

    """
    if epoch > 0:
        return str(datetime.fromtimestamp(epoch))
    return ""


def inverseDateFormatter(date: str) -> str:
    """!
    @brief Cette fonction permet de convertir une date au format "yyyy-mm-jj" en un timestamp

    Paramètres : 
        @param date : str => La date 
    Retour de la fonction : 
        @return str => le timestamp

    """
    return int(parser.parse(date).timestamp())


def secondsToTime(s: int) -> str:
    """!
    @brief Cette fonction permet de convertir un temps en seconde en un horaire de la forme "h:m:s"

    Paramètres : 
        @param s : int => le temps en seconde à convertir
    Retour de la fonction : 
        @return str => l'horaire

    """
    hour : int = s // 3600
    s : int = s % 3600
    minute: int  = s // 60
    s = s % 60
    return f'{hour}:{minute}:{s}'


def xlabels(tmps: List[int]) -> List[str]:
    """!
    @brief Cette fonction permet de transformer une liste de timestamps en une liste de date compréhensible et sans doublon

    Paramètres : 
        @param tmps : List[int] => Liste de timestamps
    Retour de la fonction : 
        @return List[str] => Liste de dates

    """
    l : List[str] = []
    for i in list(tmps):
        if i not in l:
            l.append(dateFormatter(i))
        else:
            l.append("")
    return l


def ticks(tickList: List[int]):
    """!
    @brief Cette fonction permet de réduire une liste de timestamps à 5 timestamps significatifs et de les afficher sur le graphique courant

    Paramètres : 
        @param tickList : list => Liste de timestamps

    """
    if len(tickList) > 4:
        x: List[int] = [
            tickList[0],
            tickList[int(len(tickList) * 1/4)],
            tickList[int(len(tickList) * 2/4)],
            tickList[int(len(tickList) * 3/4)],
            tickList[-1]
        ]
        plt.xticks(x, xlabels(x))
    else:
        plt.xticks(tickList, xlabels(tickList))


### Partie 1.1 : L'analyse des ports

#### Fonctions permettant de récuppérer le protocol associé à un numéro de port.

In [3]:
def _getDico():
    """!
    @brief Cette fonction permet de créer un dictionnaire à partir d'un tableau csv. Le tableau représente les numéros de ports correspondant à des protocols connus
    """
    with open('./python/portsList.csv', 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        dico: Dict[int, str] = {}  # création du dictionnaire
        for row in csv_reader:
            try:
                # Si la ligne du csv possède un port correct alors on le met dans le dictionnaire en tant que clé et son protocol en tant que valeur
                dico[int(row[1])] = row[0]
            except:
                pass
    return dico


# On sauvegarde le dictionnaire pour ne pas le regénérer à chaque appel de fonction.
_listport = _getDico()


def converter(port: str) -> str:
    """!
    @brief Cette fonction permet de récuppérer un protocol à partir d'un numéro de port.

    Paramètres : 
        @param port : str => Le numéro de port dont on souhaite connaitre le protocol
    Retour de la fonction : 
        @return str => Le protocol associé au numéro de port ou "Inconnu" si aucun n'a été trouvé

    """
    try:
        protocol: str = _listport.get(int(port), "Inconnu")
        return protocol if protocol != "" else "Inconnu"
    except:
        return "Inconnu"


#### Récuppération des donnés représentant l'activité des ports du serveur.

In [4]:
def recupperationDonnesPorts(chemin: str) -> Dict[str, Dict[int, Dict[str, str]]]:
    """!
    @brief Cette fonction permet de récupperer les données relatives à l'activité des ports et de les convertir en un Dictionnaire utilisable par les fonctions suivantes.

    Paramètres : 
        @param chemin : str => Le chemin menant au fichier csv où se trouve les données d'activité des ports.
    Retour de la fonction : 
        @return Dict[str, Dict[int, Dict[str, str]]] => Le dictionnaire représentant les données des ports. 

    """

    dico  : Dict[str, Dict[int, Dict[str, str]]] = {}
    with open(chemin, newline='') as csvfile:
        datareader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in datareader:  # pour chaque ligne du fichier csv
            """
            Les données du csv sont organisées comme suit :
            @ip,port,protocol,timestamp
            """

            # On recuppère l'adresse ip liée à l'interface
            interface: str = row[0]
            temps: int = int(row[3])  # On récuppère le timestamp
            port: str = row[1]  # On récuppère le port
            # Et on récuppère le protocol, celui-ci étant un numéro de port nous devons le convertir grace à la fonction converter(vue plus haut)
            protocol: str = converter(row[2])

            """
            Nous suivons ensuite l'algorithme suivant:

            Si l'adresse ip se trouve déjà dans les clés du dictionnaire principal
                Si le timestamp se trouve déjà dans les clés du dictionnaire associé à l'interface
                    On ajoute un nouveau couple clé valeur (port, protocol) dans le dictionnaire associé au timestamp
                Sinon
                    On crée un nouveau dictionnaire ayant déjà un couple clé valeur (port, protocol) que l'on assigne comme étant le dictionnaire associé au timestamp
            Sinon
                On crée un nouveau dictionnaire ayant pour clé le timestamp et pour valeur un autre dictionnaire avec le couple (port, protocol) que l'on assigne à l'interface 
                courante dans le dictionnaire principal.
            """

            if interface in dico:
                if temps in dico[interface]:
                    dico[interface][temps][port] = protocol
                else:
                    dico[interface][temps] = {port: protocol}
            else:
                dico[interface] = {
                    temps: {port: protocol}
                }

    return dico  # On renvoie le dictionnaire principal.


#### Fonction générant le premier graphique représentant le nombre de port ouvert sur le serveur (ou sur une interface particulière) en fonction du temps

In [5]:
def affichageNombrePort(portDict: Dict[str, Dict[int, Dict[str, str]]], interface="toutes", temps_debut: int = -1, temps_fin: int = float('inf')):
    """!
    @brief Cette fonction permet de générer un histogram représenant le nombre de port ouverts sur le serveur ou sur une interface en fonction d'une période 

    Paramètres : 
        @param portDict : Dict[str, Dict[int, Dict[str, str]]] => Le dictionnaire représentant les données d'activité des ports
        @param interface = "toutes" => L'adresse ip de l'interface à observer, par défaut toutes les interfaces seront prises en compte.
        @param temps_debut : int = -1 => Le timestamp de début de période, par défaut tous les timestamps seront compris
        @param temps_fin : int = float('inf') => Le timestamp de fin de période, par défaut tous les timestamps seront compris

    """
    times : Dict[int, int] = {}  # On prépare un dictionnaire qui nous permettra d'avoir en clé les timestamps et en valeurs le nombre de ports qui était ouvert à cet instant.
    fig, axs = plt.subplots()  # On prépare une fenêtre matplotlib

    """
    On suit l'algorithme suivant:

    Si l'interface entrée en paramètre n'existe pas dans les clés de portDict
        Pour chaque interface
            Pour chaque timestamp dans les clés de portDict[interface]
                Si le timestamp se trouve déjà dans les clés de times
                    On augmente le total se trouvant dans times[temps] par le nombre de ports se trouvant dans le dictionnaire associé au timestamp
                Sinon
                    On place le nombre de ports se trouvant dans le dictionnaire associé au timestamp dans times[temps]
    Sinon
        Pour chaque timestamp dans les clés de portDict[interface]
            Si le timestamp se trouve déjà dans les clés de times
                On augmente le total se trouvant dans times[temps] par le nombre de ports se trouvant dans le dictionnaire associé au timestamp
            Sinon
                On place le nombre de ports se trouvant dans le dictionnaire associé au timestamp dans times[temps]
    """

    if interface not in portDict:
        fig.suptitle(
            "Nombre de port ouverts par rapport au temps (toutes interfaces)", fontsize=24)
        for inter in portDict:
            for temps in portDict[inter]:
                if temps_debut <= temps and temps <= temps_fin:
                    if temps in times:
                        times[temps] += len(portDict[inter][temps])
                    else:
                        times[temps] = len(portDict[inter][temps])
    else:
        fig.suptitle(
            f"Nombre de port ouverts par rapport au temps (Interface : {interface})", fontsize=24)
        for temps in portDict[interface]:
            if temps_debut < temps and temps < temps_fin:
                if temps in times:
                    times[temps] += len(portDict[interface][temps])
                else:
                    times[temps] = len(portDict[interface][temps])

    # On crée un histogram ayant pour X les timestamps (times.keys()) et pour y les totaux (times.values)
    axs.bar(times.keys(), times.values(), width=4)
    # On demande à matplotlib d'utiliser la fonction dateFormatter pour afficher des dates compréhensible au lieu des timestamps normaux
    axs.xaxis.set_major_formatter(dateFormatter)
    fig.autofmt_xdate()  # On active la fonction de matplotlib permettant d'afficher en diagonale les dates pour faciliter la lecture


#### Fonction générant le deuxième Graphique représentant l'état d'un numéro de port sur le serveur(ou bien sur une interface particulière) en fonction d'une période 

In [6]:
def affichageActivitePort(portDict: Dict[str, Dict[int, Dict[str, str or None]]], port: str, interface: str = "toutes", temps_debut: int = -1, temps_fin: int = float('inf')):
    """!
    @brief Cette fonction permet d'afficher un graphique montrant l'état d'un port

    Paramètres : 
        @param portDict : Dict[str,Dict[int,Dict[str,strorNone]]] => Le dictionnaire représentant les données d'activité des ports
        @param port : str => le numéro du port à observer
        @param interface : str = "toutes" => l'adresse ip de l'interface sur laquelle observer le port, par défaut toutes les interfaces seront affichées.
        @param temps_debut : int = -1 => Le timestamp de début de période, par défaut tous les timestamps seront compris
        @param temps_fin : int = float('inf') => Le timestamp de fin de période, par défaut tous les timestamps seront compris

    """

    fig, axs = plt.subplots()  # On prépare une fenêtre matplotlib
    
    if temps_debut == temps_fin:
        temps_fin += 24*3600

    
    # On récuppère la liste des adresses ip des interfaces.
    inters: List[str] = list(portDict.keys())

    """
    On suit l'algorithme suivant:

    Si l'interface entrée en paramètre n'existe pas dans les clés de portDict
        On crée une liste tmps qui contiendra les timestamps et une liste interfaces qui contiendra les id des interfaces pour chaque timestamp
        Pour chaque interface dans portDict
            Pour chaque timestamp dans portDict[interface]
                Si le numéro de port se trouve dans le dictionnaire portDict[interface][timestamp]
                    On ajoute le timestamp dans tmps
                    On ajoute l'identifiant de l'interface dans interfaces
        On génère un nuage de points. à partir des timestamps en X et des id d'interfaces en y (tmps, interfaces)
    Sinon
        On change la liste inters pour qu'elle ne contienne que l'adresse ip de l'interface sélectionnée
        On créer une liste times qui contiendra les timestamps qui correspondent au port ouvert
        Pour chaque timestamp dans portDict[interface]
                Si le numéro de port se trouve dans le dictionnaire portDict[interface][timestamp]
                    On ajoute le timestamp dans tmps
        On génère un nuage de points. à partir des timestamps en X et en Y on génère une liste de 0 de la même taille que times
    """
    if interface not in portDict:
        fig.suptitle(
            f"Affichage de l'état du port {port} pour chaque interface", fontsize=24)
        tmps : List[int] = []
        interfaces : List[int]= []
        for inter in portDict:
            for temps in portDict[inter]:
                if temps_debut <= temps and temps <= temps_fin:
                    if port in portDict[inter][temps]:
                        tmps.append(temps)
                        interfaces.append(inters.index(inter))
        axs.scatter(tmps, interfaces)

        ticks(tmps)
        plt.yticks(interfaces, [inters[i] for i in interfaces])

    else:
        fig.suptitle(
            f"Affichage de l'état du port {port} sur l'interface {interface}", fontsize=24)
        inters = [interface]
        times : List[int]= []
        for temps in portDict[interface]:
            if temps_debut <= temps and temps <= temps_fin:
                if port in portDict[interface][temps]:
                    times.append(temps)
        axs.scatter(times, [0 for _ in times])

        ticks(times)
        plt.yticks([0 for _ in times], [interface for _ in times])

    def interFormatter(id, _) -> str:
        """!
        @brief Cette fonction permet de formater l'affichage en y du graph. Celui-ci affichera les adresses ip des interfaces au lieu des nombres normaux.

        Paramètres : 
            @param id => un nombre qui correspond à l'index de l'adresse ip dans la liste inters 
            @param _ => Un paramètre qui est requis par matplotlib mais non utilisé par la fonction
        Retour de la fonction : 
            @return str => Une adresse ip correspondant à une interface ou une chaine vide

        """
        if id in [float(i) for i in range(len(inters))]:
            return inters[int(id)]
        else:
            return ""

    # On demande à matplotlib d'utiliser la fonction interFormatter pour afficher des adresses ip d'interface au lieu des nombres par défaut
    axs.yaxis.set_major_formatter(interFormatter)
    fig.autofmt_xdate()  # On active la fonction de matplotlib permettant d'afficher en diagonale les dates pour faciliter la lecture


#### Fonction générant le troisème Graphique représentant la proportion des protocols utilisés sur le serveur(ou bien sur une interface particulière) en fonction d'une période 

In [7]:
def affichageProportionProtocols(portDict: Dict[str, Dict[int, Dict[str, str]]], interface: str = "toutes", temps_debut: int = -1, temps_fin: int = float('inf')):
    """!
    @brief Cette fonction permet de générer un diagramme circulaire représentant la proportion des protocols utilisés sur le serveur

    Paramètres : 
        @param portDict : Dict[str,Dict[int,Dict[str,str]]] => Le dictionnaire représentant les données d'activité des ports
        @param interface : str = "toutes" => l'adresse ip de l'interface sur laquelle observer le port, par défaut toutes les interfaces seront affichées.
        @param temps_debut : int = -1 => Le timestamp de début de période, par défaut tous les timestamps seront compris
        @param temps_fin : int = float('inf') => Le timestamp de fin de période, par défaut tous les timestamps seront compris

    """

    proto: Dict[str, int] = {}

    if temps_debut == temps_fin:
        temps_fin += 24*3600

    # On récuppère le nombre d'occurences de chaque protocol en fonction de l'interface sélectionnée et de la période sélectionnée
    if interface not in portDict:
        plt.suptitle(
            "Proportion des protocols utilisés sur toutes les interfaces")
        for inter in portDict:
            for temps in portDict[inter]:
                if temps_debut <= temps and temps <= temps_fin:
                    for port in portDict[inter][temps]:
                        protocol: str = portDict[inter][temps][port]
                        if protocol in proto:
                            proto[protocol] += 1
                        else:
                            proto[protocol] = 1
    else:
        plt.suptitle(
            f"Proportion des protocols utilisés sur l'interface {interface}")
        for temps in portDict[interface]:
            if temps_debut <= temps and temps <= temps_fin:
                for port in portDict[interface][temps]:
                    protocol: str = portDict[interface][temps][port]
                    if protocol in proto:
                        proto[protocol] += 1
                    else:
                        proto[protocol] = 1

    # On transfome notre dictionnaire en deux listes qui seront plus facile à utiliser
    values : List[int] = list(proto.values())
    labels : List[str] = list(proto.keys())

    # On trie les listes de façons à avoir les proportions de protocols dans l'ordre décroissant
    labels = sorted(
        labels, key=lambda x: values[labels.index(x)], reverse=True)
    values.sort(reverse=True)

    # Petite fonction qui servira à formatter l'affichage des pourcentages
    def autopct(x): return f'{x:.2f}%'

    # Afin d'éviter une surcharge du graphique, nous limitons l'affichage à 5 valeurs pertinantes ou moins.
    if len(values) > 5:
        # On récuppère les 4 protocols les plus utilisés et on ajoute un nouveau label "Autres"
        x : List[str] = labels[:5]
        x.append("Autres")
        # On récuppère les 4 valeurs les plus grandes et on ajoute une nouvelle valeur qui est la somme de toutes les autres valeurs
        y : List[int] = values[:5]
        y.append(sum(values[5:]))
        # On construit le diagramme
        plt.pie(
            y,
            labels=x,
            startangle=0,
            labeldistance=1.25,
            autopct=autopct
        )
    else:
        # On construit le diagramme directement grâce aux valeurs des listes.
        plt.pie(
            values,
            labels=labels,
            startangle=0,
            labeldistance=1.25,
            autopct=autopct
        )


### Partie 1.2 : L'analyse de la connectivité

#### Récuppération des données représentant la connectivité du serveur 

In [8]:
def recupperationDonnesConnectivite(chemin: str) -> Dict[int, List[float]]:
    """!
    @brief la fonction permet de récupérer 2 fichier csv puis de les mettre sous forme d'un Dict[int, List[float]]

    Paramètres : 
        @param chemin : str => chemin du fichier csv de la metrique 1
    Retour de la fonction : 
        @return Dict[int, List[float]] => Dictionnaire représentant les données de connectivité

    """

    # Dictionnaire qui représentera les données de connectivité
    connectDict: Dict[int, List[int]] = {}
    up: int  # débit montant en Kilo octet par seconde
    down: int  # débit descendant en Kilo octet par seconde
    tps: int  # Le timestamp
    pg: int  # durée du ping en milisecondes

    # On ouvre le fichier csv contenant les données et on le lit
    # Il est organisé comme suit : ping, download, upload, timestamp
    with open(chemin, newline='') as csvfile:
        datareader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in datareader:
            up : float = float(row[2])
            down : float = float(row[1])
            pg : float = float(row[0])
            tps : int= int(row[3])
            # On ajoute dans le dictionnaire le couple suivant
            # clé = timestamp, valeur = List[ping,download,upload]
            connectDict[tps] = [pg,down,up]

    return connectDict  # on retourne le dictionnaire


#### Fonctions générant le premier graphique représentant les statistiques de connectivité sur une période

In [9]:
def affichageStatistiqueConnectivite(connectDict: Dict[str, List[int]], mode="Tous", temps_debut: int = -1, temps_fin: int = float('inf')):
    """!
    @brief Cette fonction permet d'afficher un graphique montrant données de connectivité en fonction du temps

    Paramètres : 
        @param connectDict : Dict[str,List[int]] => dictionnaire représentant les données de connectivité
        @param mode = "tous" => Chaine de caractère qui permet de choisir quelles données afficher. (toutes, ping, upload, download)
        @param temps_debut : int = -1 => Le timestamp de début de période, par défaut tous les timestamps seront compris
        @param temps_fin : int = float('inf') => Le timestamp de fin de période, par défaut tous les timestamps seront compris

    """

    def graphique(axs: plt.Axes, x: List[int], y: List[int], titre: str):
        #On place les points
        axs.scatter(x, y)
        #On trace une ligne rouge en poitiée entre les points 
        axs.plot(x, y, 'r:')
        #On met un titre au graphique
        axs.set_title(titre)
        # On demande à matplotlib d'utiliser la fonction dateFormatter pour afficher des dates compréhensible au lieu des timestamps normaux
        axs.xaxis.set_major_formatter(dateFormatter)

    y: List[int] = []
    x: List[int] = []

    if temps_debut == temps_fin:
        temps_fin += 24*3600

    titres : List[str] = [
        "Ping en ms en fonction du temps",
        "Download en Ko/s en fonction du temps", 
        "Upload en Ko/s en fonction du temps"
    ]

    #On vérifie si on doit afficher toutes les mesures ou seulement une en particulier.
    if mode != "Tous":
        #Dans le cas ou c'est une en particulier : 

        modes : List[str] = ["Ping", "Download", "Upload"]
        mode_id : int = modes.index(mode)
        #On récupère l'index de la mesure ce qui nous permettra d'accèder aux données la concernant dans le dictionnaire connectDict
        for i in connectDict:
            #On prend en compte la période demandée
            if temps_debut <= i and i <= temps_fin:
                #On ajoute dans la liste y les valeurs et en x le timestamp
                y.append(connectDict[i][mode_id])
                x.append(i)

        #On prépare l'affichage de matplotlib
        fig, axs = plt.subplots()

        #On génère le graphique 
        graphique(axs, x, y, titres[mode_id])
    else:
        #Dans le cas où on veut l'ensemble des graphiques :

        #On prépare l'affichage de matplotlib en le séparant en 3 graphiques.
        fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 6))

        i: int = 0
        for ax in axes:
            #Pour chaque mesure on va récuppèrer les données dans le dictionnaire de la même manière qu'au dessus nous allons en faire un graphique
            y.clear()
            x.clear()
            for tmp in connectDict:
                if temps_debut <= tmp and tmp <= temps_fin:
                    y.append(connectDict[tmp][i])
                    x.append(tmp)
            graphique(ax, x, y, titres[i])
            i += 1
    fig.autofmt_xdate()  # On active la fonction de matplotlib permettant d'afficher en diagonale les dates pour faciliter la lecture


#### Fonctions générant le deuxième graphique représentant les moyennes des différentes mesures de connectivité sur une période

In [10]:
def affichageHistogrammeMoyennes(connectdict: Dict[str, List[int]], temps_debut: int = -1, temps_fin: int = float('inf')):
    """!
    @brief Cette fonction permet de générer deux histogrammes. Le premier représentant l'upload et le download et le deuxième le ping

    Paramètres : 
        @param connectdict : Dict[str,List[int]] => dictionnaire représentant les données de connectivité
        @param temps_debut : int = -1 => Le timestamp de début de période, par défaut tous les timestamps seront compris
        @param temps_fin : int = float('inf') => Le timestamp de fin de période, par défaut tous les timestamps seront compris

    """

    if temps_debut == temps_fin:
        temps_fin += 24*3600

    # On prépare l'affichage matplotlib
    fig, axes = plt.subplots(nrows=1, ncols=2, gridspec_kw={
                             'width_ratios': [4, 1]})

    # On déclare les variables qui nous seront utiles
    titres: List[str] = ["Ping", "Download", "Upload"]
    y: List[int] = [0, 0, 0]
    compteur: List[int] = [0, 0, 0]

    # On récuppère toutes les mesures sur la période concernée
    for tmp in connectdict:
        if temps_debut <= tmp and tmp <= temps_fin:
            for i in range(3):
                y[i] += connectdict[tmp][i]
                compteur[i] += 1

    # Premier graphique : Upload et Download

    # On isole les mesures d'upload et de download et on fait leurs moyennes.
    ry: List[float] = [y[i]/compteur[i] for i in range(2) if compteur[i] != 0]

    # On vérifie qu'on a bien des valeurs à afficher sinon on affiche un histogramme vide
    if len(ry) > 0:
        axes[0].bar(
            range(len(ry)),
            ry
        )
    else:
        axes[0].bar(
            [0, 1],
            [0, 0]
        )
    # On configure l'affichage
    axes[0].yaxis.grid(True)
    plt.sca(axes[0])

    # On remplace les valeurs arbitraire des index par les noms de nos mesures
    plt.xticks(range(len(titres)-1), titres[:2])

    plt.ylabel("kilo octet/s")
    axes[0].set_title("Upload / Download")

    # On génère l'affichage des valeurs sur les barres de l'histogramme
    for index, value in enumerate(ry):
        axes[0].text(
            index,
            value//2,
            f'{value:.5}',
            ha="center",
            color="black",
            weight="bold",
            size="medium"
        )

    # Second Graphique : Ping

    # On effectue la moyenne des pings, si il n'y en a pas on met le plus grand entier possible pour signifier la perte de connectivité
    if compteur[2] != 0:
        ry = y[2]/compteur[2]
    else:
        ry = sys.maxsize

    # On génère l'histogramme
    axes[1].bar(0, ry)

    # On configure l'affichage
    axes[1].yaxis.grid(True)
    plt.sca(axes[1])

    plt.xticks([0], ["Ping"])
    plt.ylabel("millisecondes")
    axes[1].set_title("Moyenne des pings")
    plt.subplots_adjust(wspace=0.4, top=.7)

    # On génère l'affichage des valeurs sur les barres de l'histogramme
    axes[1].text(
        0,
        ry//2,
        f'{float(ry):.5}',
        ha="center",
        color="black",
        weight="bold",
        size="medium"
    )

    # On récuppère le premier et dernier timestamp qui seront utilisé pour montrer la période choisie
    p_deb : str = dateFormatter(list(connectDict.keys())[0])
    p_fin : str = dateFormatter(list(connectDict.keys())[-1])

    # Si l'utilisateur à spécifier une autre période alors utilise les paramètres temps_debut et temps_fin
    if temps_debut != -1:
        p_deb = dateFormatter(temps_debut)
    if temps_fin != float('inf'):
        p_fin = dateFormatter(temps_fin)

    # On affiche le titre avec la période choisie.
    plt.suptitle(
        f"Moyennes des différentes mesures\n Sur la période {p_deb} -> {p_fin}", fontsize=18)

## Partie 2 : L'outil de supervision

### Importations des données depuis les fichiers csv

In [11]:
chemin_port: str = "./python/portData.csv"
chemin_connect: str = "./python/connectivityData.csv"
portDict: Dict[str, Dict[int, Dict[str, str]]
               ] = recupperationDonnesPorts(chemin_port)
connectDict: Dict[int, List[int]
                  ] = recupperationDonnesConnectivite(chemin_connect)


### Mise en place des fonctions permettant d'afficher des éléments dynamiques sur le notebook jupyter

In [12]:
#Liste d'horaires qui seront utile pour l'affichage des sliders (éléments permettant de choisir le temps de la période)
timeList : List[str] = [secondsToTime(i) for i in range(24*3600)]

#Afin d'avoir une cohérence dans tous les affichages dynamiques, nous mettons en place un layout simple
app_layout : widgets.Layout = widgets.Layout(display='flex',
                            flex_flow='row nowrap',
                            align_items='center',
                            border='none',
                            width='100%',
                            margin='5px 5px 5px 5px')


def commonWidgets(dropDownDescription: str, dropDownOptions: List[str], buttonDescription: str = "Afficher") -> List[widgets.Widget]:
    """!
    @brief Cette fonction permet de générer les widgets dynamiques qui sont utilisés pour la plupart des affichages

    Paramètres : 
        @param dropDownDescription : str => le Label du dropdown
        @param dropDownOptions : List[str] => Les options du dropdown
        @param buttonDescription : str = "Afficher" => le label du bouton
    Retour de la fonction : 
        @return List[widgets.Widget] => La liste des widgets

    """

    # Le dropdown
    dropdown = widgets.Dropdown(
        options=dropDownOptions,
        value=dropDownOptions[0],
        description=dropDownDescription,
    )

    # Les calendrier et slider pour le choix de période
    deb = widgets.DatePicker(
    )
    deb_label = widgets.Label(
        'Choisissez une date de début de période'
    )
    deb_picker = widgets.SelectionSlider(
        options=timeList,
        index=0,
        value=timeList[0]
    )
    fin = widgets.DatePicker(
    )
    fin_label = widgets.Label(
        'Choisissez une date de fin de période'
    )
    fin_picker = widgets.SelectionSlider(
        options=timeList,
        index=len(timeList)-1,
        value=timeList[len(timeList)-1]
    )
    Hbox = widgets.HBox([
        widgets.VBox([deb_label, deb, deb_picker]),
        widgets.VBox([fin_label, fin, fin_picker]),
    ])

    # Le bouton d'affichage
    button = widgets.Button(
        description=buttonDescription,
        button_style='info',
        icon='check'
    )

    # La zone d'affichage Hors cellule
    output = widgets.Output()

    return [dropdown, deb, deb_picker, fin, fin_picker, Hbox, button, output]


def notebookNombrePort(portDict: Dict[str, Dict[int, Dict[str, str]]]) -> widgets.Box:
    """!
    @brief Cette fonction permet de générer l'ensemble des éléments qui permettrons de visualiser le nombre de port sur le serveur

    Paramètres : 
        @param portDict : Dict[str,Dict[int,Dict[str,str]]] => Le dictionnaire représentant les données d'activité des ports
    Retour de la fonction : 
        @return widgets.Box => Un Widget qui contient l'ensemble des éléments dynamiques

    """
    interfaces : List[str] = list(portDict.keys())
    interfaces.insert(0, "Toutes")

    dropdown, deb, deb_picker, fin, fin_picker, Hbox, button_nombrePort, output = commonWidgets(
        "Interface :", interfaces)

    def func_button_nombrePort(_):
        with output:
            clear_output()
            if deb.value != None and fin.value != None:
                print(deb_picker.value)
                affichageNombrePort(portDict, dropdown.value, inverseDateFormatter(
                    f'{deb.value} {deb_picker.value}'), inverseDateFormatter(f'{fin.value} {fin_picker.value}'))
            else:
                affichageNombrePort(portDict, dropdown.value)
            plt.show()

    button_nombrePort.on_click(func_button_nombrePort)

    Vbox : widgets.VBox = widgets.VBox([dropdown, Hbox, button_nombrePort, output])

    app : widgets.Box = widgets.Box([Vbox], layout=app_layout)
    return app


def notebookActivitePort(portDict: Dict[str, Dict[int, Dict[str, str]]]) -> widgets.Box:
    """!
    @brief Cette fonction permet de générer l'ensemble des éléments qui permettrons de visualiser l'activité d'un port sur le serveur

    Paramètres : 
        @param portDict : Dict[str,Dict[int,Dict[str,str]]] => Le dictionnaire représentant les données d'activité des ports
    Retour de la fonction : 
        @return widgets.Box => Un Widget qui contient l'ensemble des éléments dynamiques

    """
    interfaces : List[str] = list(portDict.keys())
    interfaces.insert(0, "Toutes")

    dropdown, deb, deb_picker, fin, fin_picker, Hbox2, button_etatPort, output = commonWidgets(
        "Interface :", interfaces, "Afficher")

    port : widgets.BoundedIntText = widgets.BoundedIntText(
        value=22,
        min=0,
        max=65535,
        step=1,
        description='n°port:',
    )

    Hbox1 : widgets.HBox = widgets.HBox([dropdown, port])

    def func_button_etatPort(_):
        with output:
            clear_output()
            if deb.value != None and fin.value != None:
                affichageActivitePort(portDict, str(port.value), dropdown.value, inverseDateFormatter(
                    f'{deb.value} {deb_picker.value}'), inverseDateFormatter(f'{fin.value} {fin_picker.value}'))
            else:
                affichageActivitePort(
                    portDict, str(port.value), dropdown.value)
            plt.show()

    button_etatPort.on_click(func_button_etatPort)

    Vbox : widgets.VBox = widgets.VBox([Hbox1, Hbox2, button_etatPort, output])

    app : widgets.Box = widgets.Box([Vbox], layout=app_layout)
    return app


def notebookProportionProtocols(portDict: Dict[str, Dict[int, Dict[str, str]]]) -> widgets.Box:
    """!
    @brief Cette fonction permet de générer l'ensemble des éléments qui permettrons de visualiser la proportion des protocols utilisés sur le serveur

    Paramètres : 
        @param portDict : Dict[str,Dict[int,Dict[str,str]]] => Le dictionnaire représentant les données d'activité des ports
    Retour de la fonction : 
        @return widgets.Box => Un Widget qui contient l'ensemble des éléments dynamiques

    """
    interfaces : List[str] = list(portDict.keys())
    interfaces.insert(0, "Toutes")

    dropdown, deb, deb_picker, fin, fin_picker, Hbox, button_proportionProtocol, output = commonWidgets(
        "Interface :", interfaces, "Afficher")

    def func_button_proportionProtocol(_):
        with output:
            clear_output()
            if deb.value != None and fin.value != None:
                print(deb_picker.value)
                affichageProportionProtocols(portDict, dropdown.value, inverseDateFormatter(
                    f'{deb.value} {deb_picker.value}'), inverseDateFormatter(f'{fin.value} {fin_picker.value}'))
            else:
                affichageProportionProtocols(portDict, dropdown.value)
            plt.show()

    button_proportionProtocol.on_click(func_button_proportionProtocol)

    Vbox : widgets.VBox = widgets.VBox([dropdown, Hbox, button_proportionProtocol, output])

    app : widgets.Box = widgets.Box([Vbox], layout=app_layout)
    return app


def notebookStatistiquesConnectivite(connectDict: Dict[str, List[int]]) -> widgets.Box:
    """!
    @brief Cette fonction permet de générer l'ensemble des éléments qui permettrons de visualiser la connectivité du serveur

    Paramètres : 
        @param connectDict : Dict[str,List[int]] => Le dictionnaire représentant les données de connectivité
    Retour de la fonction : 
        @return widgets.Box => Un Widget qui contient l'ensemble des éléments dynamiques

    """

    modes : List[str] = ["Tous", "Upload", "Download", "Ping"]

    dropdown, deb, deb_picker, fin, fin_picker, Hbox, button_statistiques, output = commonWidgets(
        "Mode :", modes, "Afficher")

    def func_button_statistiques(_):
        with output:
            clear_output()
            if deb.value != None and fin.value != None:
                print(deb_picker.value)
                affichageStatistiqueConnectivite(connectDict, dropdown.value, inverseDateFormatter(
                    f'{deb.value} {deb_picker.value}'), inverseDateFormatter(f'{fin.value} {fin_picker.value}'))
            else:
                affichageStatistiqueConnectivite(connectDict, dropdown.value)
            plt.show()

    button_statistiques.on_click(func_button_statistiques)

    Vbox : widgets.VBox = widgets.VBox([dropdown, Hbox, button_statistiques, output])

    app : widgets.Box = widgets.Box([Vbox], layout=app_layout)
    return app


def notebookMoyennesConnectivite(connectDict: Dict[str, List[int]]) -> widgets.Box:
    """!
    @brief Cette fonction permet de générer l'ensemble des éléments qui serviront à la représentation des moyennes de connectivité.

    Paramètres : 
        @param connectDict : Dict[str,List[int]] => Le dictionnaire représentant les données de connectivité
    Retour de la fonction : 
        @return widgets.Box => Un Widget qui contient l'ensemble des éléments dynamiques

    """
    modes : List[str] = ["Tous", "Upload", "Download", "Ping"]

    _, deb, deb_picker, fin, fin_picker, Hbox, button_statistiques, output = commonWidgets(
        "Mode :", modes, "Afficher")

    def func_button_statistiques(_):
        with output:
            clear_output()
            if deb.value != None and fin.value != None:
                print(deb_picker.value)
                affichageHistogrammeMoyennes(connectDict, inverseDateFormatter(
                    f'{deb.value} {deb_picker.value}'), inverseDateFormatter(f'{fin.value} {fin_picker.value}'))
            else:
                affichageHistogrammeMoyennes(connectDict)
            plt.show()

    button_statistiques.on_click(func_button_statistiques)

    Vbox : widgets.VBox = widgets.VBox([Hbox, button_statistiques, output])

    app : widgets.Box = widgets.Box([Vbox], layout=app_layout)
    return app


actualisation_port_button = widgets.Button(
    description='Actualiser',
    button_style="success"
)


def actualiser_port(_):
    global portDict
    global connectDict
    portDict = recupperationDonnesPorts(chemin_port)
    portDict = recupperationDonnesPorts(chemin_connect)


actualisation_port_button.on_click(actualiser_port)

actualisation_port_button


Button(button_style='success', description='Actualiser', style=ButtonStyle())

## Supervision du nombre de port ouverts sur le serveur

In [13]:
display(notebookNombrePort(portDict))


Box(children=(VBox(children=(Dropdown(description='Interface :', options=('Toutes', '0.0.0.0', '192.162.70.239…

## Supervision de l'état d'un port sur le serveur

In [14]:
display(notebookActivitePort(portDict))


Box(children=(VBox(children=(HBox(children=(Dropdown(description='Interface :', options=('Toutes', '0.0.0.0', …

## Supervision de la proportion des protocols sur le serveur

In [15]:
display(notebookProportionProtocols(portDict))


Box(children=(VBox(children=(Dropdown(description='Interface :', options=('Toutes', '0.0.0.0', '192.162.70.239…

## Supervision des statistiques de connectivité

In [16]:
display(notebookStatistiquesConnectivite(connectDict))


Box(children=(VBox(children=(Dropdown(description='Mode :', options=('Tous', 'Upload', 'Download', 'Ping'), va…

## Supervision des moyennes de connectivité

In [17]:
display(notebookMoyennesConnectivite(connectDict))


Box(children=(VBox(children=(HBox(children=(VBox(children=(Label(value='Choisissez une date de début de périod…